# Computer Vision Analysis
#### By Ronny Toribio, Kadir O. Altunel, Michael Cook-Stahl
#### Based on [Hands on Machine Learning 2nd edition](https://github.com/ageron/handson-ml2/) and [FER2013](https://www.kaggle.com/code/ritikjain00/model-training-fer-13)

### Import modules and declare constants

In [ ]:
%matplotlib inline
import os.path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

BATCH_SIZE = 32
INPUT_SHAPE = (224, 224, 1)

### Load Facial Emotion Recognition dataset
#### training, validation, and testing

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.2, horizontal_flip=True, validation_split=0.10)
Xy_train = train_datagen.flow_from_directory(os.path.join("fer2013", "train"), batch_size=BATCH_SIZE, 
                                          target_size=(224, 224), shuffle=True, subset="training",
                                          color_mode="grayscale", class_mode="categorical")

Xy_valid = train_datagen.flow_from_directory(os.path.join("fer2013", "train"), batch_size=BATCH_SIZE, 
                                          target_size=(224, 224), shuffle=True, subset="validation",
                                          color_mode="grayscale", class_mode="categorical")

test_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.3)
Xy_test = test_datagen.flow_from_directory(os.path.join("fer2013", "test"), batch_size=BATCH_SIZE,
                                          target_size=(224, 224), shuffle=True,
                                          color_mode="grayscale", class_mode="categorical")

### Build CNN model

In [ ]:
model = Sequential([
    # Convolution Block 1
    Conv2D(32, kernel_size=(3, 3), padding="same", activation="relu", input_shape=INPUT_SHAPE),
    Conv2D(64, kernel_size=(3, 3), padding="same", activation="relu"),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="valid"),
    Dropout(0.25),
    
    # Convolution Block 2
    Conv2D(128, kernel_size=(3, 3), padding="same", activation="relu", kernel_regularizer=l2(0.01)),
    Conv2D(256, kernel_size=(3, 3), padding="valid", activation="relu", kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), padding="valid"),
    Dropout(0.25),
    
    # Classification Block
    Flatten(),
    Dense(1024, activation="relu"),
    Dropout(0.5),
    Dense(7, activation="softmax")
])

### Model properties

In [ ]:
model.layers

In [ ]:
model.summary()

In [ ]:
# requires pydot and graphviz modules
plot_model(model, "cnn_model.png", show_shapes=True)

### Compile model

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.0001, decay=1e-6),
              metrics=["accuracy"])

### Checkpoint callback

In [ ]:
checkpoint_cb = ModelCheckpoint("cnn_model.h5", save_best_only=True)

### Early stopping callback

In [ ]:
early_stopping_cb = EarlyStopping(patience=10, restore_best_weights=True)

### Reduce learning rate on plateau callback

In [ ]:
reduce_lr_cb = ReduceLROnPlateau(patience=3, factor=0.01)

### Train the model

In [ ]:
history = model.fit(Xy_train, epochs=60, validation_data=(Xy_valid),
                    steps_per_epoch=Xy_train.n // BATCH_SIZE,
                    validation_steps=Xy_valid.n // BATCH_SIZE,
                    callbacks=[checkpoint_cb, early_stopping_cb, reduce_lr_cb])

### Roll back to the best model

In [ ]:
model = load_model("cnn_model.h5")

### Plot training history

In [ ]:
adjusted_epoch = [x - 0.5 for x in history.epoch]
fig, ax = plt.subplots(1, 1, figsize=(8, 5))
ax.grid(True)
ax.plot(history.epoch, history.history["loss"], "b-", label="loss")
ax.plot(history.epoch, history.history["accuracy"], "r-", label="accuracy")
ax.plot(adjusted_epoch, history.history["val_loss"], "c-", label="val_loss")
ax.plot(adjusted_epoch, history.history["val_accuracy"], "g-", label="val_accuracy")
ax.legend()
fig.savefig("cnn_model_training.png")
plt.show()

### Evaluate model

In [ ]:
model.evaluate(Xy_test)

### Save weights

In [ ]:
model.save_weights("cnn_model_weights.ckpt")